In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os, shutil
from belief_tree import Tree

### Initialise the agent

In [2]:
def policy(q_values, temp=None, policy_type='softmax'):

    '''
    ----
    Agent's policy

    q_values -- q values at the current state
    temp     -- inverse temperature
    type     -- softmax / greeedy
    ----
    '''

    if np.all(q_values == 0):
        return np.array([0.5, 0.5])

    if temp:
        t = temp
    else:
        t = 1
        
    if policy_type == 'softmax':
        return np.exp(q_values*t)/np.sum(np.exp(q_values*t))
    elif policy_type == 'greedy':
        return np.array(q_values >= q_values.max()).astype(int)
    else:
        raise KeyError('Unknown policy type')

In [3]:
M = np.array([
    [29, 1],
    [1, 1]
])

gamma = 0.9
xi    = 0.0

Q = np.zeros(2)

tree = Tree(M, Q, 1, 'greedy')

### Full Bayesian updates

In [4]:
horizon = 3
tree.root_q_values = Q
tree.build_tree(horizon)
tree.full_updates(gamma)

In [5]:
qval_tree = tree.qval_tree
qval_tree[0][(0, 0, 0)]

array([2.61966667, 2.153     ])

### Replay

In [6]:
horizon = 3
tree.root_q_values = Q
tree.build_tree(horizon)
qval_history, need_history, replays = tree.replay_updates(gamma, xi)

In [7]:
qval_tree = tree.qval_tree
qval_tree[0][(0, 0, 0)]

array([2.61966667, 0.        ])

### Generate replay tree

In [ ]:
from tex_tree import generate_big_tex_tree

save_folder = '/home/georgy/Documents/Dayan_lab/PhD/bandits/Data/example_tree/seq/'
if os.path.exists(save_folder):
    shutil.rmtree(save_folder)
    os.mkdir(save_folder)
else:
    os.makedirs(save_folder)

for idx, rep in enumerate(replays):
    these_replays = replays[:idx+1]
    save_path = os.path.join(save_folder, 'tex_tree_%u.tex'%idx)
    generate_big_tex_tree(horizon, these_replays, qval_history[idx], need_history[idx], save_path)

### Distribution updates asymmetry 

In [ ]:
from scipy.stats import beta
from scipy.special import kl_div

In [ ]:
x = np.linspace(0.001, 1, 100)

a1b, b1b = np.array([2, 1])
rv1b = beta(a1b, b1b)  

a1a, b1a = np.array([3, 1])
rv1a = beta(a1a, b1a)

a1aa, b1aa = np.array([4, 1])
rv1aa = beta(a1aa, b1aa)

In [ ]:
plt.figure(figsize=(10, 3))
plt.subplot(131)
plt.plot(x, rv1b.pdf(x))
plt.title(r'$\alpha=2, \beta=1$')

plt.subplot(132)
plt.plot(x, rv1a.pdf(x))
plt.title(r'$\alpha=3, \beta=1$')

print(np.sum(kl_div(rv1b.pdf(x), rv1a.pdf(x))))

plt.subplot(133)
plt.plot(x, rv1aa.pdf(x))
plt.title(r'$\alpha=4, \beta=1$')

print(np.sum(kl_div(rv1a.pdf(x), rv1aa.pdf(x))))

plt.tight_layout()

# plt.savefig('/home/georgy/Documents/Dayan_lab/PhD/bandits/Data/betas.png')

### Number of replays until convergence

In [24]:
save_folder = '/home/georgy/Documents/Dayan_lab/PhD/bandits/bandit/data/convergence/num_replays/'

# vary these parameters
alphas_0 = np.arange(1, 40)
betas_0  = np.arange(1, 40)

# constant parameters
alpha_1 = 1
beta_1  = 1

gamma  = 0.9
xi     = 0.0
xi_cap = 0.001

for horizon in [3, 4]:

    R = np.full((len(alphas_0), len(betas_0)), np.nan)

    for alpha_0 in alphas_0:
        for beta_0 in betas_0:
            
            num_replays = 0

            # prior belie at the root
            M = np.array([
                [alpha_0, beta_0],
                [alpha_1, beta_1],
            ])

            # MF Q values at the root
            Q    = np.zeros(2)

            # initialise the agent
            tree = Tree(M, Q, 80, 'softmax')

            # build the tree
            tree.build_tree(horizon)
            
            # do full bayesian updates
            tree.full_updates(gamma)
            qval_tree = tree.qval_tree
            qvals     = qval_tree[0][(0, 0, 0)]
            print(r'$\alpha_0=%u, \beta_0=%u$'%(alpha_0, beta_0))
            v_full = np.max(qvals)
            
            # do replay
            tree.root_q_values = np.zeros(2)
            qval_history, need_history, replays = tree.replay_updates(gamma, xi)
            for qval_tree_idx in range(len(qval_history)):
                qval_tree = qval_history[qval_tree_idx]
                qvals     = qval_tree[0][(0, 0, 0)]
                v_replay  = np.max(qvals)
                if np.abs(v_replay - v_full) <= xi_cap: # converged after this many replays
                    # print(v_replay, v_full, tree.xi)
                    R[alpha_0-alphas_0[0], beta_0-betas_0[0]] = qval_tree_idx
                    break
    
    file_name = 'horizon%u_alpha%u_beta%u'%(horizon, alpha_1, beta_1)
    np.save(os.path.join(save_folder, 'data', file_name + '.npy'), R)
    # R = np.load(os.path.join(save_folder, 'data', file_name + '.npy'))

    fig = plt.figure(figsize=(10, 8), dpi=100, constrained_layout=True)
    # plt.contourf(R, vmax=np.nanmax(R))
    plt.pcolormesh(R, vmax=np.nanmax(R))
    cbar = plt.colorbar()
    cbar.ax.set_ylabel('Number of replays', rotation=270, fontsize=14, labelpad=18)
    plt.xlabel(r'$\beta_0$', fontsize=14)
    plt.xticks(range(R.shape[0]), range(1, R.shape[0]+1))
    plt.ylabel(r'$\alpha_0$', fontsize=14)
    plt.yticks(range(R.shape[1]), range(1, R.shape[1]+1))
    plt.title(r'$\alpha_1=%u, \beta_1=%u$'%(alpha_1, beta_1), fontsize=18)

    file_name = 'horizon%u_alpha%u_beta%u'%(horizon-1, alpha_1, beta_1)
    # np.save(os.path.join(save_folder, 'data', file_name + '.npy'), R)
    plt.savefig(os.path.join(save_folder, file_name + '.png'))

    plt.close()

    print('Done with horizon %u'%horizon)

$\alpha_0=1, \beta_0=1$
$\alpha_0=1, \beta_0=2$
$\alpha_0=1, \beta_0=3$
$\alpha_0=1, \beta_0=4$
$\alpha_0=1, \beta_0=5$
$\alpha_0=1, \beta_0=6$
$\alpha_0=1, \beta_0=7$
$\alpha_0=1, \beta_0=8$
$\alpha_0=1, \beta_0=9$
$\alpha_0=1, \beta_0=10$
$\alpha_0=1, \beta_0=11$
$\alpha_0=1, \beta_0=12$
$\alpha_0=1, \beta_0=13$
$\alpha_0=1, \beta_0=14$
$\alpha_0=1, \beta_0=15$
$\alpha_0=1, \beta_0=16$
$\alpha_0=1, \beta_0=17$
$\alpha_0=1, \beta_0=18$
$\alpha_0=1, \beta_0=19$
$\alpha_0=1, \beta_0=20$
$\alpha_0=1, \beta_0=21$
$\alpha_0=1, \beta_0=22$
$\alpha_0=1, \beta_0=23$
$\alpha_0=1, \beta_0=24$
$\alpha_0=1, \beta_0=25$
$\alpha_0=1, \beta_0=26$
$\alpha_0=1, \beta_0=27$
$\alpha_0=1, \beta_0=28$
$\alpha_0=1, \beta_0=29$
$\alpha_0=1, \beta_0=30$
$\alpha_0=1, \beta_0=31$
$\alpha_0=1, \beta_0=32$
$\alpha_0=1, \beta_0=33$
$\alpha_0=1, \beta_0=34$
$\alpha_0=1, \beta_0=35$
$\alpha_0=1, \beta_0=36$
$\alpha_0=1, \beta_0=37$
$\alpha_0=1, \beta_0=38$
$\alpha_0=1, \beta_0=39$
$\alpha_0=2, \beta_0=1$
$\alpha_0=

### Replay threshold and value convergence

In [ ]:
save_folder = '/home/georgy/Documents/Dayan_lab/PhD/bandits/bandit/data/convergence/xi'

# vary these parameters
xis       = np.append(0, np.logspace(np.log2(0.001), np.log2(1.0), 18, base=2))
horizons  = [2, 3, 4, 5]
betas     = [0.1, 0.5, 1, 2, 4, 8]

# constant parameters
alpha_0 = 10
beta_0  = 6
alpha_1 = 3
beta_1  = 2

gamma   = 0.9

# prior belie at the root
M = np.array([
    [alpha_0, beta_0],
    [alpha_1, beta_1],
], dtype=int)

# MF Q values at the root
Q    = np.zeros(2)

# initialise the agent
tree = Tree(M, Q, 1, 'softmax')

# store results here
R      = np.zeros((len(horizons), len(betas), len(xis)))
nreps  = np.zeros((len(horizons), len(betas), len(xis)), dtype=int)
R_true = np.zeros(len(horizons))

for hidx, horizon in enumerate(horizons):
            
    num_replays = 0

    # build the tree
    tree.build_tree(horizon)
    
    # do full bayesian updates
    tree.full_updates(gamma)
    qval_tree    = tree.qval_tree
    qvals        = qval_tree[0][(0, 0, 0)]
    v_full       = np.max(qvals)
    R_true[hidx] = v_full
    
    for bidx, beta in enumerate(betas):

        tree.policy_temp = beta

        for xidx, xi in enumerate(xis):

            # do replay
            tree.root_q_values = np.zeros(2)
            qval_history, need_history, replays = tree.replay_updates(gamma, xi)
            qval_tree = tree.qval_tree
            qvals     = qval_tree[0][(0, 0, 0)]
            # v_replay  = np.max(qvals)
            v_replay  = np.dot(policy(qvals, temp=beta), qvals)
            R[hidx, bidx, xidx]     = v_replay
            nreps[hidx, bidx, xidx] = len(replays)-1

    print('Horizon %u'%horizon)

fig = plt.figure(figsize=(14, 18), dpi=100, constrained_layout=True)

for hidx, h in enumerate(horizons):
    
    plt.subplot(2, 2, hidx+1)

    for bidx, beta in enumerate(betas): 

        # plt.subplot(1, len(betas), bidx+1)
        
        plt.plot(R[hidx, bidx, ::-1], label='Beta %.1f'%beta)
        plt.scatter(range(len(xis)), R[hidx, bidx, ::-1])
        
        # for x in range(len(xis)):
        #     plt.text(x-0.03, R[hidx, bidx, len(xis)-x-1]+0.04, nreps[hidx, bidx, len(xis)-x-1])

        if bidx == (len(betas) - 1):

            plt.axhline(R_true[hidx], linestyle='--', color='k', alpha=0.7, label='Optimal value')
        
            plt.legend(prop={'size': 9})
            plt.ylabel('Root value', fontsize=12)
            plt.ylim(0, np.max(R_true)+0.05)

            plt.title(r'$\alpha_0=%u, \beta_0=%u, \alpha_1=%u, \beta_1=%u$'%(alpha_0, beta_0, alpha_1, beta_1), fontsize=12)

            plt.xlabel(r'$\xi$', fontsize=12)
            plt.xticks(range(R.shape[2]), ['%.4f'%i for i in xis[::-1]], rotation=60)

file_name = 'alpha0%u_beta0%u_alpha1%u_beta1%u_all'%(alpha_0, beta_0, alpha_1, beta_1)
np.save(os.path.join(save_folder, 'data', file_name + '.npy'), R)
plt.savefig(os.path.join(save_folder, file_name + '.png'))

plt.close()



